<a href="https://colab.research.google.com/github/n1lays1ngh/Delhi-Air-Quality-Prediction/blob/main/Data_Formatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

In [3]:
data_folder_path = '/content/drive/MyDrive/Pollutant Data'

# Check if the folder exists to make sure the path is correct
if os.path.exists(data_folder_path):
    print(f"Success! Folder found at: {data_folder_path}")
    print("You can now run Cell 2.")
else:
    print(f"Error: Folder not found at '{data_folder_path}'")
    print("Please double-check the folder name (it's case-sensitive) and path.")

Success! Folder found at: /content/drive/MyDrive/Pollutant Data
You can now run Cell 2.


In [4]:
import pandas as pd
import os # This is needed for os.path.join

# --- Configuration ---

# Define the 12 months in order. This is crucial for melting.
MONTHS_ORDER = [
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
]

# Define the years you have data for
YEARS = range(2017, 2026) # Goes from 2017 up to 2025

# --- Helper Function to Process Each File ---

def process_data_file(filepath, year, data_type_name):
    """
    Reads one of your CSV files, unpivots it, and formats it.

    filepath: Path to the data file (e.g., 'aqi_2017.csv')
    year: The integer year for this data
    data_type_name: The name for the value column ('AQI' or 'Pollutants')
    """
    try:
        # --- 1. Read the Data ---
        df = pd.read_csv(filepath)

        # --- 2. Rename 'Day' column (or confirm it) ---
        if 'Date' in df.columns:
            df = df.rename(columns={'Date': 'Day'})

        # --- 3. Unpivot (Melt) the Data ---
        df_melted = df.melt(
            id_vars=['Day'],
            value_vars=MONTHS_ORDER,
            var_name='Month',
            value_name=data_type_name
        )

        # --- 4. Create the Full Date Column ---
        df_melted['Year'] = year

        # Combine Day, Month, and Year into a single string
        date_str = df_melted['Day'].astype(str) + '-' + \
                     df_melted['Month'] + '-' + \
                     df_melted['Year'].astype(str)

        # Convert the string into a proper datetime object
        df_melted['Date'] = pd.to_datetime(date_str, format='%d-%B-%Y', errors='coerce')

        # --- 5. Clean and Finalize ---
        df_melted = df_melted.dropna(subset=['Date'])
        final_df = df_melted[['Date', data_type_name]]

        return final_df

    except FileNotFoundError:
        print(f"Warning: File not found, skipping: {filepath}")
        return None
    except Exception as e:
        print(f"Error processing file {filepath}: {e}")
        return None

# --- Main Script to Process All Files ---

# Retrieve the folder path defined in Cell 1
# (This assumes Cell 1 has been run)
try:
    data_folder_path
except NameError:
    print("Error: 'data_folder_path' is not defined.")
    print("Please run Cell 1 first to mount Google Drive.")
    # Stop the script if the path isn't defined
    raise

print("Starting data reformatting...")
all_processed_dfs = []

for year in YEARS:
    print(f"--- Processing Year: {year} ---")

    # Define the expected file paths using the Google Drive folder path
    aqi_file = os.path.join(data_folder_path, f'aqi_{year}.csv')
    pollutants_file = os.path.join(data_folder_path, f'pollutants_{year}.csv')

    # Process the AQI file for the year
    aqi_df = None
    if os.path.exists(aqi_file):
        aqi_df = process_data_file(aqi_file, year, 'AQI')
    else:
        print(f"No AQI file found for {year}.")

    # Process the Pollutants file (if it exists)
    pollutants_df = None
    if os.path.exists(pollutants_file):
        pollutants_df = process_data_file(pollutants_file, year, 'Pollutants')
    else:
        print(f"No Pollutants file found for {year}.")

    # --- Combine the year's data ---
    if aqi_df is not None and pollutants_df is not None:
        year_df = pd.merge(aqi_df, pollutants_df, on='Date', how='outer')
        all_processed_dfs.append(year_df)
    elif aqi_df is not None:
        all_processed_dfs.append(aqi_df)
    elif pollutants_df is not None:
        all_processed_dfs.append(pollutants_df)

# --- Final Combination ---
if all_processed_dfs:
    final_data = pd.concat(all_processed_dfs)
    final_data = final_data.sort_values(by='Date').reset_index(drop=True)

    # --- Save the Result ---
    # We save the final file back to your Google Drive folder
    output_filename = os.path.join(data_folder_path, 'delhi_aqi_timeseries_formatted.csv')
    final_data.to_csv(output_filename, index=False)

    print("\n--- Success! ---")
    print(f"All data processed and saved to: '{output_filename}'")

    print("\nFirst 5 rows of formatted data:")
    print(final_data.head())

    print("\nLast 5 rows of formatted data:")
    print(final_data.tail())

    print("\nData Info:")
    final_data.info()
else:
    print("\nNo data was processed. Please check your file paths and configuration.")

Starting data reformatting...
--- Processing Year: 2017 ---
No Pollutants file found for 2017.
--- Processing Year: 2018 ---
No Pollutants file found for 2018.
--- Processing Year: 2019 ---
No Pollutants file found for 2019.
--- Processing Year: 2020 ---
No Pollutants file found for 2020.
--- Processing Year: 2021 ---
--- Processing Year: 2022 ---
--- Processing Year: 2023 ---
--- Processing Year: 2024 ---
--- Processing Year: 2025 ---
Error processing file /content/drive/MyDrive/Pollutant Data/aqi_2025.csv: "The following id_vars or value_vars are not present in the DataFrame: ['April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']"
Error processing file /content/drive/MyDrive/Pollutant Data/pollutants_2025.csv: "The following id_vars or value_vars are not present in the DataFrame: ['April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']"

--- Success! ---
All data processed and saved to: '/content/drive/MyDrive/Poll

In [5]:
import pandas as pd
import os # This is needed for os.path.join

# --- Configuration ---

# Define the 12 months in order. This is crucial for melting.
MONTHS_ORDER = [
    'January', 'February', 'March', 'April', 'May', 'June',
    'July', 'August', 'September', 'October', 'November', 'December'
]

# Define the years you have data for
YEARS = range(2017, 2026) # Goes from 2017 up to 2025

# --- Helper Function to Process Each File ---

def process_data_file(filepath, year, data_type_name):
    """
    Reads one of your CSV files, unpivots it, and formats it.

    filepath: Path to the data file (e.g., 'aqi_2017.csv')
    year: The integer year for this data
    data_type_name: The name for the value column ('AQI' or 'Pollutants')
    """
    try:
        # --- 1. Read the Data ---
        df = pd.read_csv(filepath)

        # --- 2. Rename 'Day' column (or confirm it) ---
        if 'Date' in df.columns:
            df = df.rename(columns={'Date': 'Day'})

        # --- 3. Unpivot (Melt) the Data ---

        # --- THIS IS THE FIX ---
        # Instead of using the full MONTHS_ORDER, we find which months
        # from that list *actually exist* as columns in the current file.
        months_to_melt = [month for month in MONTHS_ORDER if month in df.columns]

        # If we didn't find any valid month columns, skip the file.
        if not months_to_melt:
            print(f"Warning: No valid month columns found in {filepath}. Skipping.")
            return None

        # Now, we melt *only* the months that were found.
        df_melted = df.melt(
            id_vars=['Day'],
            value_vars=months_to_melt, # <-- This list is now dynamic
            var_name='Month',
            value_name=data_type_name
        )
        # --- END OF FIX ---

        # --- 4. Create the Full Date Column ---
        df_melted['Year'] = year

        # Combine Day, Month, and Year into a single string
        date_str = df_melted['Day'].astype(str) + '-' + \
                     df_melted['Month'] + '-' + \
                     df_melted['Year'].astype(str)

        # Convert the string into a proper datetime object
        df_melted['Date'] = pd.to_datetime(date_str, format='%d-%B-%Y', errors='coerce')

        # --- 5. Clean and Finalize ---
        df_melted = df_melted.dropna(subset=['Date'])
        final_df = df_melted[['Date', data_type_name]]

        return final_df

    except FileNotFoundError:
        print(f"Warning: File not found, skipping: {filepath}")
        return None
    except Exception as e:
        print(f"Error processing file {filepath}: {e}")
        return None

# --- Main Script to Process All Files ---

# Retrieve the folder path defined in Cell 1
try:
    data_folder_path
except NameError:
    print("Error: 'data_folder_path' is not defined.")
    print("Please run Cell 1 first to mount Google Drive.")
    # Stop the script if the path isn't defined
    raise

print("Starting data reformatting...")
all_processed_dfs = []

for year in YEARS:
    print(f"--- Processing Year: {year} ---")

    # Define the expected file paths using the Google Drive folder path
    aqi_file = os.path.join(data_folder_path, f'aqi_{year}.csv')
    pollutants_file = os.path.join(data_folder_path, f'pollutants_{year}.csv')

    # Process the AQI file for the year
    aqi_df = None
    if os.path.exists(aqi_file):
        aqi_df = process_data_file(aqi_file, year, 'AQI')
    else:
        print(f"No AQI file found for {year}.")

    # Process the Pollutants file (if it exists)
    pollutants_df = None
    if os.path.exists(pollutants_file):
        pollutants_df = process_data_file(pollutants_file, year, 'Pollutants')
    else:
        print(f"No Pollutants file found for {year}.")

    # --- Combine the year's data ---
    if aqi_df is not None and pollutants_df is not None:
        year_df = pd.merge(aqi_df, pollutants_df, on='Date', how='outer')
        all_processed_dfs.append(year_df)
    elif aqi_df is not None:
        all_processed_dfs.append(aqi_df)
    elif pollutants_df is not None:
        all_processed_dfs.append(pollutants_df)

# --- Final Combination ---
if all_processed_dfs:
    final_data = pd.concat(all_processed_dfs)
    final_data = final_data.sort_values(by='Date').reset_index(drop=True)

    # --- Save the Result ---
    # We save the final file back to your Google Drive folder
    output_filename = os.path.join(data_folder_path, 'delhi_aqi_timeseries_formatted2.csv')
    final_data.to_csv(output_filename, index=False)

    print("\n--- Success! ---")
    print(f"All data processed and saved to: '{output_filename}'")

    print("\nFirst 5 rows of formatted data:")
    print(final_data.head())

    print("\nLast 5 rows of formatted data:")
    print(final_data.tail())

    print("\nData Info:")
    final_data.info()
else:
    print("\nNo data was processed. Please check your file paths and configuration.")

Starting data reformatting...
--- Processing Year: 2017 ---
No Pollutants file found for 2017.
--- Processing Year: 2018 ---
No Pollutants file found for 2018.
--- Processing Year: 2019 ---
No Pollutants file found for 2019.
--- Processing Year: 2020 ---
No Pollutants file found for 2020.
--- Processing Year: 2021 ---
--- Processing Year: 2022 ---
--- Processing Year: 2023 ---
--- Processing Year: 2024 ---
--- Processing Year: 2025 ---

--- Success! ---
All data processed and saved to: '/content/drive/MyDrive/Pollutant Data/delhi_aqi_timeseries_formatted2.csv'

First 5 rows of formatted data:
        Date    AQI Pollutants
0 2017-01-01  345.0        NaN
1 2017-01-02  337.0        NaN
2 2017-01-03  331.0        NaN
3 2017-01-04  381.0        NaN
4 2017-01-05  321.0        NaN

Last 5 rows of formatted data:
           Date    AQI      Pollutants
3007 2025-03-27  259.0      PM10,PM2.5
3008 2025-03-28  205.0  NO2,OZONE,PM10
3009 2025-03-29  153.0    CO,NO2,OZONE
3010 2025-03-30  138.0    